# Load packages

In [1]:
# !pip uninstall -y kfp
!pip install kfp

# ! pip install git+https://github.com/kubeflow/pipelines.git#subdirectory=sdk/python

In [2]:
import kfp
import json
import os
from kfp.onprem import use_k8s_secret
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler
from pathlib import Path

kfp.__version__

'1.6.2'

# Enter your gateway loadbalancer and the token from the cookie
[Use this extension on chrome to get token]( https://chrome.google.com/webstore/detail/editthiscookie/fngmhnnpilhplaeedifhccceomclgfbg?hl=en)

![image.png](./image.png)

In [3]:
#Update values for the load balancer and auth session
HOST='http://istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="authservice_session=MTYyMTU3ODU5NnxOd3dBTkVkTFNGTlpWMFJhVmxkV1MwdFFNMUJFV0VvM1UwUTJORVJCUkZoSlVWaEtNbEJhVGpSUU0wRTNTVXRKUkU5V1VVY3pWRkU9fD8Nvz8zvysM595tORbkrCV_AhbburhYEH_QAtlf0nLu"
namespace="kubeflow-user-example-com"

In [4]:
client = kfp.Client(host=HOST+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace=namespace)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 4, 22, 8, 44, 39, tzinfo=tzlocal()),
 'description': None,
 'id': 'aac96a63-616e-4d88-9334-6ca8df2bb956',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

### Using dsl for model archiver and torchserve integration
#### Init container is used for mar file generation

In [5]:
DEPLOY="bertserve"
MODEL="bert"

In [6]:
prepare_tensorboard_op = load_component_from_file("./examples/bert/yaml/tensorboard/component.yaml")
prep_op = components.load_component_from_file(
    "./examples/bert/yaml/pre_process/component.yaml"
)
train_op = components.load_component_from_file(
    "./examples/bert/yaml/train/component.yaml"
)
deploy_op = load_component_from_file("./examples/bert/yaml/deploy/component.yaml")

In [7]:
minio_op = components.load_component_from_file(
    "./examples/bert/yaml/minio/component.yaml"
)

In [8]:
@dsl.pipeline(name="Training pipeline", description="Sample training job test")
def pytorch_bert(
    minio_endpoint="http://minio-service.kubeflow:9000",
    log_bucket="mlpipeline",
    log_dir=f"tensorboard/logs/{dsl.RUN_ID_PLACEHOLDER}",
    mar_path=f"mar/{dsl.RUN_ID_PLACEHOLDER}/model-store",
    config_prop_path=f"mar/{dsl.RUN_ID_PLACEHOLDER}/config",
    model_uri=f"s3://mlpipeline/mar/{dsl.RUN_ID_PLACEHOLDER}",
    tf_image="jagadeeshj/tb_plugin:v1.8",
    deploy="bertserve",
    namespace="kubeflow-user-example-com",
    confusion_matrix_log_dir=f"confusion_matrix/{dsl.RUN_ID_PLACEHOLDER}/",
    num_samples=1000
):

    prepare_tb_task = prepare_tensorboard_op(
        log_dir_uri=f"s3://{log_bucket}/{log_dir}",
        image=tf_image,
        pod_template_spec=json.dumps(
            {
                "spec": {
                    "containers": [
                        {
                            "env": [
                                {
                                    "name": "AWS_ACCESS_KEY_ID",
                                    "valueFrom": {
                                        "secretKeyRef": {
                                            "name": "mlpipeline-minio-artifact",
                                            "key": "accesskey",
                                        }
                                    },
                                },
                                {
                                    "name": "AWS_SECRET_ACCESS_KEY",
                                    "valueFrom": {
                                        "secretKeyRef": {
                                            "name": "mlpipeline-minio-artifact",
                                            "key": "secretkey",
                                        }
                                    },
                                },
                                {"name": "AWS_REGION", "value": "minio"},
                                {"name": "S3_ENDPOINT", "value": f"{minio_endpoint}"},
                                {"name": "S3_USE_HTTPS", "value": "0"},
                                {"name": "S3_VERIFY_SSL", "value": "0"},
                            ]
                        }
                    ]
                }
            }
        ),
    ).set_display_name("Visualization")

    prep_task = prep_op().after(prepare_tb_task).set_display_name("Preprocess & Transform")
    train_task = (
        train_op(
            input_data=prep_task.outputs["output_data"],
            profiler="pytorch",
            confusion_matrix_url=f"minio://{log_bucket}/{confusion_matrix_log_dir}",
            num_samples=num_samples
        )
        .apply(
            use_k8s_secret(
                secret_name="mlpipeline-minio-artifact",
                k8s_secret_key_to_env={
                    "secretkey": "MINIO_SECRET_KEY",
                    "accesskey": "MINIO_ACCESS_KEY",
                },
            )
        )
        .after(prep_task)
        .set_display_name("Training")
    )

    minio_tb_upload = (
        minio_op(
            bucket_name="mlpipeline",
            folder_name=log_dir,
            input_path=train_task.outputs["tensorboard_root"],
            filename="",
        )
        .apply(
            use_k8s_secret(
                secret_name="mlpipeline-minio-artifact",
                k8s_secret_key_to_env={
                    "secretkey": "MINIO_SECRET_KEY",
                    "accesskey": "MINIO_ACCESS_KEY",
                },
            )
        )
        .after(train_task)
        .set_display_name("Tensorboard Events Pusher")
    )
    minio_mar_upload = (
        minio_op(
            bucket_name="mlpipeline",
            folder_name=mar_path,
            input_path=train_task.outputs["checkpoint_dir"],
            filename="bert_test.mar",
        )
        .apply(
            use_k8s_secret(
                secret_name="mlpipeline-minio-artifact",
                k8s_secret_key_to_env={
                    "secretkey": "MINIO_SECRET_KEY",
                    "accesskey": "MINIO_ACCESS_KEY",
                },
            )
        )
        .after(train_task)
        .set_display_name("Mar Pusher")
    )
    minio_config_upload = (
        minio_op(
            bucket_name="mlpipeline",
            folder_name=config_prop_path,
            input_path=train_task.outputs["checkpoint_dir"],
            filename="config.properties",
        )
        .apply(
            use_k8s_secret(
                secret_name="mlpipeline-minio-artifact",
                k8s_secret_key_to_env={
                    "secretkey": "MINIO_SECRET_KEY",
                    "accesskey": "MINIO_ACCESS_KEY",
                },
            )
        )
        .after(train_task)
        .set_display_name("Conifg Pusher")
    )

    model_uri = str(model_uri)
    isvc_yaml = """
    apiVersion: "serving.kubeflow.org/v1beta1"
    kind: "InferenceService"
    metadata:
      name: {}
      namespace: {}
    spec:
      predictor:
        serviceAccountName: sa
        pytorch:
          storageUri: {}
          resources:
            limits:
              memory: 4Gi   
    """.format(
        deploy, namespace, model_uri
    )
    deploy_task = (
        deploy_op(action="apply", inferenceservice_yaml=isvc_yaml)
        .after(minio_mar_upload)
        .set_display_name("Deployer")
    )


In [9]:
# Compile pipeline
compiler.Compiler().compile(pytorch_bert, 'pytorch.tar.gz', type_check=True)

/opt/conda/lib/python3.8/site-packages/kfp/components/_data_passing.py:169: UserWarning: Missing type name was inferred as "Integer" based on the value "1000".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [10]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-bert', 'pytorch.tar.gz')

### Wait for inference service below to go to `READY True` state.

In [11]:
!kubectl get isvc $DEPLOY

NAME        URL                                                      READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                 AGE
bertserve   http://bertserve.kubeflow-user-example-com.example.com   True           100                              bertserve-predictor-default-mw6x4   25h


# Inference

In [12]:
!kubectl get isvc $DEPLOY -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3

bertserve.kubeflow-user-example-com.example.com


In [13]:
S_HOSTNAME=!kubectl get isvc $DEPLOY  -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3
SERVICE_HOSTNAME=S_HOSTNAME[0]
SERVICE_HOSTNAME

'bertserve.kubeflow-user-example-com.example.com'

# Prediction

In [14]:
!curl -v -H "Host: $SERVICE_HOSTNAME" -H "Cookie: $AUTH" "$HOST/v1/models/$MODEL:predict" -d @./examples/bert/sample.txt > bert_prediction_output.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:predict HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMTU3ODU5NnxOd3dBTkVkTFNGTlpWMFJhVmxkV1MwdFFNMUJFV0VvM1UwUTJORVJCUkZoSlVWaEtNbEJhVGpSUU0wRTNTVXRKUkU5V1VVY3pWRkU9fD8Nvz8zvysM595tORbkrCV_AhbburhYEH_QAtlf0nLu
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
} [84 bytes data]
* upload completely sent off: 84 out of 84 bytes
100    84    0     0  100    84      0     69  0:00:01  0:00:01 --:--:--    69* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-leng

In [15]:
! cat bert_prediction_output.json

{"predictions": ["\"Sci/Tech\""]}

In [16]:
!curl -v -H "Host: $SERVICE_HOSTNAME" -H "Cookie: $AUTH" "$HOST/v1/models/$MODEL:explain" -d @./examples/bert/sample.txt  > bert_explaination_output.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:explain HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMTU3ODU5NnxOd3dBTkVkTFNGTlpWMFJhVmxkV1MwdFFNMUJFV0VvM1UwUTJORVJCUkZoSlVWaEtNbEJhVGpSUU0wRTNTVXRKUkU5V1VVY3pWRkU9fD8Nvz8zvysM595tORbkrCV_AhbburhYEH_QAtlf0nLu
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
} [84 bytes data]
* upload completely sent off: 84 out of 84 bytes
100    84    0     0  100    84      0      1  0:01:24  0:00:53  0:00:31     0* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-leng

In [17]:
! cat bert_explaination_output.json

{"explanations": [{"words": ["[CLS]", "bloomberg", "has", "reported", "on", "the", "economy", "[SEP]"], "importances": [0.49801128883150986, -0.04228800481977043, -0.22690382385299707, 0.1557326112787248, 0.08676915006156335, 0.17918951470597463, 0.5255293995528972, -0.5988652369043819], "delta": 0.12079699740117719}]}

In [18]:
import json
explanations_json = json.loads(open("./bert_explaination_output.json", "r").read())
explanations_json

{'explanations': [{'words': ['[CLS]',
    'bloomberg',
    'has',
    'reported',
    'on',
    'the',
    'economy',
    '[SEP]'],
   'importances': [0.49801128883150986,
    -0.04228800481977043,
    -0.22690382385299707,
    0.1557326112787248,
    0.08676915006156335,
    0.17918951470597463,
    0.5255293995528972,
    -0.5988652369043819],
   'delta': 0.12079699740117719}]}

In [19]:
prediction_json = json.loads(open("./bert_prediction_output.json", "r").read())

In [20]:
import torch
attributions = explanations_json["explanations"][0]['importances']
tokens = explanations_json["explanations"][0]['words']
delta = explanations_json["explanations"][0]['delta']

attributions = torch.tensor(attributions)
pred_prob = 0.75
pred_class = prediction_json["predictions"][0]
true_class = "Business"
attr_class ="world"

In [21]:
from captum.attr import visualization
vis_data_records =[]
vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred_prob,
                            pred_class,
                            true_class,
                            attr_class,
                            attributions.sum(),       
                            tokens,
                            delta))

In [22]:
visualization.visualize_text(vis_data_records)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Business,"""Sci/Tech"" (0.75)",world,0.58,[CLS] bloomberg has reported on the economy [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Business,"""Sci/Tech"" (0.75)",world,0.58,[CLS] bloomberg has reported on the economy [SEP]


## Clean up Script

In [23]:
! kubectl delete pod --field-selector=status.phase==Succeeded -n $namespace
! kubectl delete --all isvc -n $namespace


Error: flag needs an argument: 'n' in -n
See 'kubectl delete --help' for usage.
